In [131]:
import json
import requests
import time
import yaml
from IPython.display import clear_output
from pprint import pprint

from ai_api_client_sdk.ai_api_v2_client import AIAPIV2Client
from ai_api_client_sdk.models.artifact import Artifact
from ai_api_client_sdk.models.status import Status
from ai_api_client_sdk.models.target_status import TargetStatus
from ai_api_client_sdk.models.input_artifact_binding import InputArtifactBinding

In [132]:
# Setup
aic_service_key_path = '../resources/aic_service_key.json'
git_setup_file_path = "../resources/git_setup.json"
docker_secret_file_path = '../resources/docker_secret.json'
resource_group = "restaurant-inspections"
s3_service_key_path = '../resources/s3_service_key.json'
training_workflow_file = '../workflows/train.yaml'
serving_workflow_file = '../workflows/serve.yaml'
connection_name = "default"
path_prefix = "app"

# Manage the ML lifecycle in the cloud using AI Core and AI Core Python SDK

### Connect to your AI Core instance

In [133]:
with open(aic_service_key_path) as ask:
    aic_service_key = json.load(ask)

# AI API client that talks to the AI Core instance.
ai_api_client = AIAPIV2Client(
    base_url = aic_service_key["serviceurls"]["AI_API_URL"] + "/v2", # The present AI API version is 2
    auth_url=  aic_service_key["url"] + "/oauth/token",
    client_id = aic_service_key['clientid'],
    client_secret = aic_service_key['clientsecret']
)

### Onboard your Git repository that contains the templates

In [135]:
with open(git_setup_file_path) as gs:
		setup_json = json.load(gs)

repo_json = setup_json["repo"]

response = ai_api_client.rest_client.post(
		path="/admin/repositories",
		body={
				"name": repo_json["name"],
				"url": repo_json["url"],
				"username": repo_json["username"],
				"password": repo_json["password"]
		}
)
print(response)

AIAPIServerException: Failed to post /admin/repositories: Repository is already onboarded

### Register an application

In [136]:
app_json = setup_json["app"]
response = ai_api_client.rest_client.post(
		path="/admin/applications",
		body={
				"applicationName": app_json["applicationName"],
				"repositoryUrl": app_json["repositoryUrl"],
				"revision": app_json["revision"],
				"path": app_json["path"]
		}
)

AIAPIServerException: Failed to post /admin/applications: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"applications.argoproj.io \"465f1721-aicore-restaurant-inspections-regression\" already exists","reason":"AlreadyExists","details":{"name":"465f1721-aicore-restaurant-inspections-regression","group":"argoproj.io","kind":"applications"},"code":409}


### Register docker secret

In [137]:
with open(docker_secret_file_path) as dsf:
    docker_secret = json.load(dsf)

response = ai_api_client.rest_client.post(
    path="/admin/dockerRegistrySecrets",
    body={
        "name": docker_secret["name"],
        "data": docker_secret["data"]
    }
)
print(response)

{'message': 'Secret exists'}


### Create a resource group

In [ ]:
ai_api_client.rest_client.post(
    path="/admin/resourceGroups",
    body={
        "resourceGroupId": resource_group
    }
)

TRAINING DATASET NEEDS TO BE UPLOADED TO AMAZON S3 SEPARATELY, FOR EXAMPLE VIA AWS CLI.  
See: https://developers.sap.com/tutorials/ai-core-aiapi-clientsdk-resources.html#45dcbe5b-a94f-461e-b9eb-54f2d965e930

### Register secret to access dataset in Amazon S3 and create resource group

In [138]:
with open(s3_service_key_path) as s3sk:
    s3_service_key = json.load(s3sk)

default_secret = {
    "name": connection_name, # Name of the connection.
    "type": "S3",
    "endpoint": s3_service_key["host"],
    "bucket": s3_service_key["bucket"],
    "pathPrefix": path_prefix,
    "region": s3_service_key["region"],
    "data": {
        "AWS_ACCESS_KEY_ID": s3_service_key["access_key_id"],
        "AWS_SECRET_ACCESS_KEY": s3_service_key["secret_access_key"]
    }
}

ai_api_client.rest_client.post(
    path="/admin/objectStoreSecrets",
    body = default_secret,
    resource_group = resource_group
)

KeyboardInterrupt: 

DOCKER IMAGES STORED IN code/train AND code/infer NEED TO BE BUILT AND PUSHED TO DOCKER HUB
THROUGH THE DOCKER CLI.  
See: https://developers.sap.com/tutorials/ai-core-aiapi-clientsdk-workflows.html#f824a41d-efe8-4883-8238-caef4ac5f789

In [139]:
# Create another AI API client to use different base url.
ai_api_v2_client = AIAPIV2Client(
    base_url=aic_service_key["serviceurls"]["AI_API_URL"] + "/v2/lm",
    auth_url=aic_service_key["url"] + "/oauth/token",
    client_id=aic_service_key['clientid'],
    client_secret=aic_service_key['clientsecret'],
    resource_group=resource_group)

### Register training data as artifact

In [ ]:
with open(training_workflow_file) as twf:
    training_workflow = yaml.safe_load(twf)

scenario_id = training_workflow['metadata']['labels']['scenarios.ai.sap.com/id']

# Set the artifact configuration.
artifact = {
        "name": resource_group, # Modifiable name
        "kind": Artifact.Kind.DATASET,
        # Path has to contain the connection name used above for the AWS S3 bucket.
        "url": f"ai://{connection_name}/data",
        "description": "The training data set.",
        "scenario_id": scenario_id
}

artifact_resp = ai_api_v2_client.artifact.create(**artifact)
print(f"Artifacts registered for {scenario_id} scenario!")
pprint(vars(artifact_resp)) 

assert artifact_resp.message == 'Artifact acknowledged'

### Create Training Configuration

In [140]:
input_artifact_name = training_workflow['spec']['templates'][0]['inputs']['artifacts'][0]['name']
executable_name = training_workflow['metadata']['name']

artifact_binding = {
    "key": input_artifact_name,
    "artifact_id": artifact_resp.id
}

train_configuration = {
    "name": resource_group,
    "scenario_id": scenario_id,
    "executable_id": executable_name,
    "parameter_bindings": [],
    "input_artifact_bindings": [ InputArtifactBinding(**artifact_binding) ]
}

train_config_resp = ai_api_v2_client.configuration.create(**train_configuration)
pprint(vars(train_config_resp))

assert train_config_resp.message == 'Configuration created'

print("Configuration created for running the training")

{'id': '0df9436f-98ae-41c2-835c-e6811d648850',
 'message': 'Configuration created'}
Configuration created for running the training


### Training Execution

In [146]:
execution_resp = ai_api_v2_client.execution.create(train_config_resp.id)
pprint(vars(execution_resp))

status = None
# Wait until training finished.
while status != Status.COMPLETED and status != Status.DEAD:
    # Sleep for 5 secs to avoid overwhelming the API with requests.
    time.sleep(5)
    # Clear outputs to reduce clutter.
    clear_output(wait=True)

    execution = ai_api_v2_client.execution.get(execution_resp.id)
    status = execution.status
    print('...... execution status ......', flush=True)
    print(f"Training status: {execution.status}")
    print(f"Training status details: {execution.status_details}")


if execution.status == Status.COMPLETED:
    print(f"Training complete for execution [{execution_resp.id}]!")
    output_artifact = execution.output_artifacts[0]
    output = {
        "id": output_artifact.id,
        "name": output_artifact.name,
        "url": output_artifact.url
    }
    with open('../training_output.json', 'w') as fp:
        json.dump(output, fp)

...... execution status ......
Training status: Status.COMPLETED
Training status details: None
Training complete for execution [e922976a6eeb5446]!


### Create configuration to serve the model

In [149]:
with open(serving_workflow_file) as swf:
    serving_workflow = yaml.safe_load(swf)

scenario_id = serving_workflow['metadata']['labels']['scenarios.ai.sap.com/id']
input_artifact_name = serving_workflow['spec']['inputs']['artifacts'][0]['name']
executable_name = serving_workflow['metadata']['name']

training_output = '../training_output.json'
with open(training_output) as to:
    serving_input = json.load(to)

artifact_binding = {
    "key": input_artifact_name,
    "artifact_id": serving_input["id"]
}

serve_configuration = {
    "name": f"{resource_group}-serve",
    "scenario_id": scenario_id,
    "executable_id": executable_name,
    "parameter_bindings": [],
    "input_artifact_bindings": [ InputArtifactBinding(**artifact_binding) ]
}

serve_config_resp = ai_api_v2_client.configuration.create(**serve_configuration)

assert serve_config_resp.message == 'Configuration created'

pprint(vars(serve_config_resp))
print("configuration for serving the model created")

{'id': '089efc03-6b0a-41ec-b799-fb822bdb40e0',
 'message': 'Configuration created'}
configuration for serving the model created


### Actually serve/deploy the model

In [150]:
deployment_resp = ai_api_v2_client.deployment.create(serve_config_resp.id)
pprint(vars(deployment_resp))

# Poll deployment status.
status = None
while status != Status.RUNNING and status != Status.DEAD:
    time.sleep(5)
    clear_output(wait=True)
    deployment = ai_api_v2_client.deployment.get(deployment_resp.id)
    status = deployment.status
    print('...... deployment status ......', flush=True)
    print(deployment.status)
    pprint(deployment.status_details)

    if deployment.status == Status.RUNNING:
        print(f"Deployment with {deployment_resp.id} complete!")

# Allow some time for deployment URL to get ready.
time.sleep(10)

...... deployment status ......
Status.RUNNING
None
Deployment with d31b5360276c2443 complete!


### Do an inference request

In [151]:
body = { 
    "payload": [
        {
            "business_postal_code": "94102",
            "violation_description": "Unapproved or unmaintained equipment or utensils. Moderate risk food holding temperature. Noncompliance with HAACP plan or variance. Inadequate food safety knowledge or lack of certified food safety manager."
        },
        {
            "business_postal_code": "94102",
            "violation_description": "Moderate risk food holding temperature. Noncompliance with HAACP plan or variance. Inadequate food safety knowledge or lack of certified food safety manager."
        }
    ]
}

endpoint = f"{deployment.deployment_url}/v1/models/{resource_group}:predict"
headers = {"Authorization": ai_api_v2_client.rest_client.get_token(),
           'ai-resource-group': resource_group,
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=body)

print('Inference result:', response.json())
pprint(vars(response))

Inference result: {'predictions': [[87.89493892895823, 51.28017265137158], [88.49765829031011, 58.17775078637164]]}
{'_content': b'{"predictions":[[87.89493892895823,51.28017265137158],[88.497658'
             b'29031011,58.17775078637164]]}\n',
 '_content_consumed': True,
 '_next': None,
 'connection': <requests.adapters.HTTPAdapter object at 0x7f7a00f59b50>,
 'cookies': <RequestsCookieJar[]>,
 'elapsed': datetime.timedelta(microseconds=312137),
 'encoding': 'utf-8',
 'headers': {'content-length': '94', 'content-type': 'application/json', 'date': 'Fri, 15 Jul 2022 10:58:48 GMT', 'server': 'istio-envoy', 'x-envoy-upstream-service-time': '106'},
 'history': [],
 'raw': <urllib3.response.HTTPResponse object at 0x7f7998306730>,
 'reason': 'OK',
 'request': <PreparedRequest [POST]>,
 'status_code': 200,
 'url': 'https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d31b5360276c2443/v1/models/restaurant-inspections:predict'}


### Kill deployment

In [ ]:
delete_resp = ai_api_v2_client.deployment.modify(deployment_resp.id,
                                                 target_status=TargetStatus.STOPPED)
status = None
while status != Status.STOPPED:
    time.sleep(5)
    clear_output(wait=True)
    deployment = ai_api_v2_client.deployment.get(deployment_resp.id)
    status = deployment.status
    print('...... killing deployment ......', flush=True)
    print(f"Deployment status: {deployment.status}")